## Chapter 5

Statistics Gatherer

Most of this chapter is now accomplished using pandas/numpy in Python, however we follow the book in any case as a challenge.

In [12]:
import numpy as np
from copy import deepcopy
import random
import math

In [2]:
class StatisticsMC:
    def __init__(self):
        pass
    
    def GetResultsSoFar(self):
        pass
    
    def DumpOneResult(self,result):
        pass        
  
    def clone(self):
        pass
    
    def __del__(self):
        pass   

We implement the mean: Of course we can also implement the median, mode etc.

In [3]:
class StatisticsMean(StatisticsMC):
    def __init__(self):
        self.__RunningSum = 0
        self.__PathsDone = 0
        
    def GetResultsSoFar(self): 
        Results = [[0]]
        Results[0][0] = self.__RunningSum / self.__PathsDone
        return Results
    
    def DumpOneResult(self,result):
        self.__PathsDone += 1
        self.__RunningSum += result        
        
    def clone(self):
        return deepcopy(self)
    
    def __del__(self):
        del self

In [5]:
mean = StatisticsMean()
mean.DumpOneResult(76)
mean.DumpOneResult(50)
mean.GetResultsSoFar()

[[63.0]]

We incorparate this with our previously written pricer.

In [13]:
from random import random
from math import exp, sqrt, log

In [6]:
class PayOff():
    def __init__(self):
        pass
    
    def __call__(self,Spot):
        pass    

class PayOffBridge():        
    def __del__(self):
        del self
        
    def copy(self,InnerPayOff):
        from copy import deepcopy
        self = deepcopy(InnerPayOff)
        
class PayOffCall(PayOff,PayOffBridge):
    def __init__(self,Strike_):
        self.__Strike = Strike_
        
    def __call__(self,Spot):
        return max(Spot - self.__Strike,0)
    
class VanillaOption:
    def __init__(self, ThePayOff_, Expiry_):
        self.__ThePayOff = ThePayOff_
        self.__Expiry = Expiry_
        
    def GetExpiry(self):
        return self.__Expiry
    
    def OptionPayOff(self, Spot):
        return self.__ThePayOff(Spot)

In [7]:
class ParametersInner:
    def __init__(self):
        pass
    
    def Integral(self,time1, time2):
        pass
    
    def IntegralSquare(self,time1,time2):
        pass

class Parameters:    
    def RootMeanSquare(self,time1,time2):
        total = self.Integral(time1,time2)
        return total/(time2-time1)
    
    def Mean(self,time1, time2):
        total = self.Integral(time1,time2)
        return total/(time2-time1)
    
    def copy(self,original):
        from copy import deepcopy
        self = deepcopy(InnerPayOff) 
        
    def __del__(self):
        del self    
        
class ParametersConstant(ParametersInner,Parameters):
    def __init__(self,constant):
        self.__Constant = constant
        self.__ConstantSquare = self.__Constant*self.__Constant
    
    def Integral(self, time1, time2):
        return (time2 - time1)*self.__Constant
    
    def IntegralSquare(self, time1, time2):
        return (time2 - time1)*self.__ConstantSquare   


In [15]:
def SimpleMonteCarlo5(TheOption,
                     Spot,
                     Vol,
                     r,
                     NumberOfPaths,
                     gatherer):
    
    Expiry = TheOption.GetExpiry()    
    variance = Vol.IntegralSquare(0,Expiry)
    rootVariance = sqrt(variance)
    itoCorrection = -0.5*variance
    
    movedSpot = Spot*exp(r.Integral(0,Expiry) + itoCorrection)
    discounting = exp(-r.Integral(0,Expiry))
    for i in range(NumberOfPaths):
        thisGaussian = np.random.normal()
        thisSpot = movedSpot*exp(rootVariance*thisGaussian)
        thisPayoff = TheOption.OptionPayOff(thisSpot)
        gatherer.DumpOneResult(thisPayoff*discounting)

In [16]:
gatherer = StatisticsMean()
callPayOff = PayOffCall(40)
TheOption = VanillaOption(callPayOff,0.5)
r = ParametersConstant(0.1)
Vol = ParametersConstant(0.2)
SimpleMonteCarlo5(TheOption,42,Vol,r,10000, gatherer)
results = gatherer.GetResultsSoFar()
print(f'Call price for 10000 paths: ')
for i in range(len(results)):
    for j in range(len(results[i])):
        print(results[i][j])

Call price for 10000 paths: 
4.802691563567282


Convergence Tables:

In [18]:
class ConvergenceTable(StatisticsMC):
    def __init__(self, Inner_):
        self.__Inner = Inner_
        self.__ResultsSoFar = []
        self.__StoppingPoint = 2
        self.__PathsDone = 0
    
    def clone(self):
        return deepcopy(self)
    
    def DumpOneResult(self,result):
        self.__Inner.DumpOneResult(result)
        self.__PathsDone += 1
        
        
        if self.__PathsDone == self.__StoppingPoint:
            self.__StoppingPoint *=2
            thisResult = self.__Inner.GetResultsSoFar()            
            
            for i in range(len(thisResult)):
                thisResult[i].append(self.__PathsDone)
                self.__ResultsSoFar.append(thisResult[i]) 

            
    
    def GetResultsSoFar(self):
        
        tmp = self.__ResultsSoFar 
        
        if self.__PathsDone*2 != self.__StoppingPoint:
            thisResult = self.__Inner.GetResultsSoFar()
            
            for i in range(len(thisResult)):
                thisResult[i].append(self.__PathsDone)
                tmp.append(thisResult[i]) 
                
        return tmp
                

In [19]:
gatherer = StatisticsMean()
gathererTwo = ConvergenceTable(gatherer)
callPayOff = PayOffCall(40)
TheOption = VanillaOption(callPayOff,0.5)
r = ParametersConstant(0.1)
Vol = ParametersConstant(0.2)
SimpleMonteCarlo5(TheOption,42,Vol,r,10000, gathererTwo)
results = gathererTwo.GetResultsSoFar()
print(f'Call price for 10000 paths: ')
for i in range(len(results)):
    for j in range(len(results[i])):
        print(results[i][j])

Call price for 10000 paths: 
3.124848511062993
2
2.977979205525093
4
3.8659198129741275
8
4.705512218885523
16
3.9664684850097567
32
4.870194691708492
64
4.964282591475483
128
4.516461033654419
256
4.557417670020879
512
4.730401530384945
1024
4.838829816282456
2048
4.816182359936459
4096
4.804562216215163
8192
4.772231421069234
10000


We build the above to work with the first 4 moments.

In [ ]:
class StatisticsFirstFourMoments(StatisticsMC):
    def __init__(self):
        self.__RunningSum = 0
        self.__RunningSumDifferenceSquared
        self.__RunningSumSquared = 0
        self.__PathsDone = 0
        self.__M2 = 0
        self.__M3 = 0
        self.__M4 = 0        
        
        
    def GetResultsSoFar(self): 
        Results = [[0,0,0,0]]
        Results[0][0] = self.__RunningSum / self.__PathsDone # mean
        Results[0][1] =(self.__RunningSumSquared - self.__RunningSum*self.__RunningSum/self.__PathsDone)/(self.__PathsDone-1) #sample variance
        Results[0][2] = sqrt(self.__PathsDone)*self.__M3 / (self.__M2*sqrt(self.__M2)) # skewness
        Results[0][3] = (self.__PathsDone*self.__M4) / (self.__M2*self.__M2) - 3 # kurtosis
        return Results
    
    def DumpOneResult(self,result):
        self.__PathsDone += 1
        self.__RunningSum += result
        self.__RunningSumSquared += result*result
        delta = result - self.__RunningSum/self.__PathsDone
        delta_n = delta/self.__PathsDone
        delta_n2 = delta_n*delta_n
        term1 = delta*delta_n*(self.__PathsDone - 1)
        self.__M4 = self.__M4 + term1*delta_n2*(self.__PathsDone*self.__PathsDone
                                                - 3*self.__PathsDone + 3)+ 6*delta_n2*self.__M2 -4 *delta_n*self.__M3
        self.__M3 = self.__M3 + term1 * delta_n * (self.__PathsDone - 2) - 3 * delta_n * self.__M2
        self.__M2 = self.__M2 + term1
        
    def clone(self):
        return deepcopy(self)
    
    def __del__(self):
        del self        